# 2. RAG Basic
 - Indexing
 - Retrieval
 - Generation

## `Indexing`
> 문서를 쉽게 검색할 수 있도록
- 우리가 실제로 로드하고 Retriever 를 통해 검색하고자 하는 외부 문서를 다루는 것
- Retriever 의 목표는 입력된 Query 에 따라 관련 docs 를 찾아내는 것입니다. 
- 이때, 단순한 텍스트보다 숫자 벡터를 비교하는 것이 훨씬 쉽기에 수년간 텍스트 문서를 숫자 표현으로 압축하는 다양한 방법이 개발되었습니다.

<div style="text-align: center;">
    <img src="./img/indexing.png" alt="Indexing" style="width: 60%; display: inline-block; margin: 0 2%;">
    <img src="./img/embedding.png" alt="Embedding" style="width: 30%; display: inline-block; margin: 0 2%;">
</div>

**Word to Vector**
1. 문서의 단어 빈도를 분석하여 희소 벡터를 만드는 통계적 방법
2. 최근에는 머신러닝을 이용한 임베딩 방법
    - 문서를 고정된 길이의 벡터로 압축하여 매우 강력한 검색 방법을 제공
    - 임베딩 모델은 제한된 컨텍스트 윈도우를 가지고 있어 문서를 여러 조각으로 나누고 각 조각을 벡터로 압축
    - 질문도 동일한 방식으로 임베딩하여 벡터를 생성하고, 이 벡터들을 비교하여 질문과 관련된 문서를 찾아냅니다.

**Addtional IDEA**
- 오른쪽 그림과 같이 문서를 임베딩할 때 문서가 3차원 공간의 한 점으로 투영된다고 상상할 수 있습니다.
- 이 위치는 문서의 의미나 내용에 따라 결정됩니다.
- 따라서 공간 내 가까운 위치에 있는 문서들은 유사한 의미를 포함하게 됩니다.
- 이 아이디어는 현대 벡터 스토어에서 사용되는 많은 검색 및 검색 방법의 기초가 됩니다

## `Retrieval`
<div style="text-align: center;">
    <img src="./img/retrieval.png" alt="Indexing" style="width: 40%; display: inline-block; margin: 0 2%;">
    <img src="./img/various.png" alt="Various" style="width: 50%; display: inline-block; margin: 0 2%;">
</div>

- 문서를 임베딩하여 3D 공간에 투영하고, 질문도 동일하게 임베딩하여 해당 공간에 투영합니다. 
- 그런 다음 질문 주위의 로컬 네이버후드 검색을 통해 근처에 있는 문서를 찾아냅니다 (유사한 의미를 가진 문서를 검색합니다)
- 우리는 원하는 k 개 만큼의 문서를 선택할 수 있습니다.
- 여러 임베딩 모델, 인덱스, 문서 로더 및 스플리터를 조합하여 다양한 인덱싱 및 검색 방법을 테스트할 수 있습니다.


## `Generation`

<div style="text-align: center;">
    <img src="./img/generation.png" alt="Indexing" style="width: 80%;">
</div>

- 관련된 문서 조각들을 검색 후, 프롬프트를 완성합니다.
- 이를 컨텍스트 윈도우에 넣고 답변을 생성합니다.

---

### `Embedding` & `Similarity between query and docs`

In [3]:
%%capture
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain python-dotenv bs4

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

Count Token

In [6]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

Text embedding Model

In [7]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

Cosine Similarity

In [8]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8807044730847654
